In [30]:
import requests  
import math  
import pandas as pd  
import time
import urllib
import urllib.request
from bs4 import BeautifulSoup as Bs
import random
import re
random.seed(36)

def get_proxy():
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    headers = {'User-Agent': user_agent}
    req = urllib.request.Request('https://free-proxy-list.net/', headers=headers)

    page = urllib.request.urlopen(req).read().decode('utf-8')
    soup = Bs(page, 'html.parser')

    ip_add = []
    ds = []
    divs = soup.find_all('td', attrs={'class':None})
    for i, tag in enumerate(divs):
        if i%4 == 0:
            ip_add.append(ds)
            ds = []
        ds.append(tag.text)
    
    ip = pd.DataFrame(data = ip_add, columns = ['ip','port','Country','type'])

    ran = random.randint(0,100)
    http = 'http://'+ip.iloc[ran][0]+':'+ip.iloc[ran][1]
    https = 'https://'+ip.iloc[ran][0]+':'+ip.iloc[ran][1]
    
    return http, https

def get_json(url,num,pos):  
    '''''从网页获取JSON,使用POST请求,加上头部信息''' 
    http, https = get_proxy()

    proxies = {
        'http': http,
        'https': https,
    }

    # Create the session and set the proxies.
    s = requests.Session()
    s.proxies = proxies

    # Make the HTTP request through the session.
    #r = s.get('http://www.showmemyip.com/')
    '''''从网页获取JSON,使用POST请求,加上头部信息''' 
    user_agents=['Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20130406 Firefox/23.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:18.0) Gecko/20100101 Firefox/18.0',
                   'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/533+ \(KHTML, like Gecko) Element Browser 5.0',
                   'IBM WebExplorer /v0.94', 'Galaxy/1.0 [en] (Mac OS X 10.5.6; U; en)',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
                   'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
                   'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) \Version/6.0 Mobile/10A5355d Safari/8536.25',
                   'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) \Chrome/28.0.1468.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0; TheWorld)']
    index=random.randint(0, 9)
    user_agent=user_agents[index]
    my_headers = {  
            'User-Agent':user_agent,  
            'Host':'www.lagou.com',  
            'Referer':'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput=',  
            'X-Anit-Forge-Code':'0',  
            'X-Anit-Forge-Token': 'None',  
            'X-Requested-With':'XMLHttpRequest'  
            }  

    my_data = {  
            'first': 'true',  
            'pn':num,  
            'kd':pos}  

    res = s.post(url, headers = my_headers, data = my_data)  
    # Check if the proxy was indeed used (the text should contain the proxy IP).
    res.raise_for_status()  
    res.encoding = 'utf-8'  
    # 得到包含职位信息的字典  
    page = res.json()  
    return page


def get_page_num(count):  
    '''''计算要抓取的页数'''  
    # 每页15个职位,向上取整  
    res = math.ceil(count/15)  
    # 拉勾网最多显示30页结果  
    if res > 30:  
        return 30  
    else:  
        return res  

def get_page_info(jobs_list):  
    '''''对一个网页的职位信息进行解析,返回列表'''  
    page_info_list = []  
    for n, i in enumerate(jobs_list): 
        print('现在进行到第{}个；'.format(n))
        job_info = []  
        job_info.append(i['companyFullName'])  
        job_info.append(i['companyShortName'])  
        job_info.append(i['companySize'])  
        job_info.append(i['financeStage'])  
        job_info.append(i['district'])  
        job_info.append(i['positionName'])  
        job_info.append(i['workYear'])  
        job_info.append(i['education'])  
        job_info.append(i['salary'])  
        job_info.append(i['positionAdvantage'])  
        job_info.append(i['positionId'])
        
        url = 'https://www.lagou.com/jobs/'+ str(i['positionId']) +'.html'
        user_agents=['Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20130406 Firefox/23.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:18.0) Gecko/20100101 Firefox/18.0',
                   'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/533+ \(KHTML, like Gecko) Element Browser 5.0',
                   'IBM WebExplorer /v0.94', 'Galaxy/1.0 [en] (Mac OS X 10.5.6; U; en)',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
                   'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
                   'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) \Version/6.0 Mobile/10A5355d Safari/8536.25',
                   'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) \Chrome/28.0.1468.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0; TheWorld)']
        index=random.randint(0, 9)
        user_agent=user_agents[index]
        headers = {  
                'User-Agent':user_agent,  
                'Host':'www.lagou.com',  
                'Referer':'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput=',  
                'X-Anit-Forge-Code':'0',  
                'X-Anit-Forge-Token': 'None',  
                'X-Requested-With':'XMLHttpRequest'  
                }  
        http, https = get_proxy()
        proxies = {
            'http': http,
            'https': https,
        }
        #data = urllib.parse.urlencode(dict).encode('utf-8')
        #data参数如果要传必须传bytes（字节流）类型的，如果是一个字典，先用urllib.parse.urlencode()编码。
        time_sleep = random.randint(10, 30)
        time.sleep(time_sleep)  
        try:
            s = requests.Session()
            s.proxies = proxies
            res = s.post(url, headers = headers)  
            # Check if the proxy was indeed used (the text should contain the proxy IP).
            res.raise_for_status()  
            res.encoding = 'utf-8' 
            soup = Bs(res.content, 'html.parser')
            content = soup.find('dd', class_='job_bt').get_text()
            job_info.append(content)
        except:
            print('访问网页失败！')
        finally:    
            page_info_list.append(job_info)  
    return page_info_list  

In [31]:
random.seed(36)

url = 'https://www.lagou.com/jobs/positionAjax.json?px=new&needAddtionalResult=false'
#url = 'https://www.lagou.com/jobs/positionAjax.json?city=%E6%B7%B1%E5%9C%B3&needAddtionalResult=false'  
# 先设定页数为1,获取总的职位数  
pos = input('请输入你想查阅的职位名称：')
page_1 = get_json(url,1,pos)  
total_count = page_1['content']['positionResult']['totalCount']  
num = get_page_num(total_count)  
total_info = []  
time_sleep = random.randint(10, 30)
time.sleep(time_sleep)  
print('职位总数:{},页数:{}'.format(total_count,num))  

请输入你想查阅的职位名称：AI算法工程师
职位总数:29082,页数:30


In [ ]:
#total_info = []  
for n in range(0,num+1):  
    # 对每个网页读取JSON, 获取每页数据  
    page = get_json(url,n,pos)  
    jobs_list = page['content']['positionResult']['result']  
    page_info = get_page_info(jobs_list)  
    total_info += page_info  
    print('已经抓取第{}页, 职位总数:{}'.format(n, len(total_info)))  
    # 每次抓取完成后,暂停一会,防止被服务器拉黑  
    time_sleep = random.randint(10, 30)
    time.sleep(time_sleep)  
#将总数据转化为data frame再输出  
      

现在进行到第0个；
现在进行到第1个；
现在进行到第2个；
现在进行到第3个；
现在进行到第4个；
现在进行到第5个；
现在进行到第6个；
现在进行到第7个；
现在进行到第8个；


In [ ]:
df = pd.DataFrame(data = total_info,columns = ['公司全名','公司简称','公司规模','融资阶段','区域','职位名称','工作经验','学历要求','工资','职位福利','职位id','职位描述'])   
df.to_csv('lagou_jobs_{}.csv'.format(n),index = False)  
print('已保存为csv文件.') 